## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

5. Add your OpenAI key.  OpenAI is only needed for example purposes.  If you have your own model, you can substitute those in it's place.

In [ ]:
%pip install okareo
%pip install transformers

In [ ]:
# We have a set of 15 food items that we would like the language model to explain
# We upload these texts into a scenario set, so that they can be used in a test run in the next cell
import os
from okareo import Okareo

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

scenario = okareo.upload_scenario_set(file_path='./text_generation_example.jsonl', scenario_name="Text generation")
print(scenario) # use scenario set id in output for the next step

In [ ]:
# Evaluate the scenario and model combination and then get a link to the results on Okareo
import random
import string
from transformers import pipeline

from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData
from okareo.model_under_test import CustomModel
from okareo_api_client.models.test_run_type import TestRunType

random_string = ''.join(random.choices(string.ascii_letters, k=5))
mut_name = f"Text Generation Model - {random_string}"
eval_name = f"Text Generation Run - {random_string}"
model_name = "gpt2" # Insert any model from huggingface you would like to use

class LanguageGenerationModel(CustomModel):
    initial_phrase = "Hi, I am a fruit and vegetable expert."
    generator = pipeline('text-generation', model=model_name, max_length=100)

    # Since we are using this model locally, we create a custom model class to invoke GPT-2
    def invoke(self, input_value):        
        output_text = self.generator(f"{self.initial_phrase} {input_value} are")[0]['generated_text']
        return output_text[len(self.initial_phrase) + 1:], "Model response: success"

model_under_test = okareo.register_model(
    name=mut_name,
    model=LanguageGenerationModel(
        name=model_name
    ),
)

evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=scenario,
    test_run_type=TestRunType.NL_GENERATION,
    calculate_metrics=True,
    metrics_kwargs={
        "dimensions": ["conciseness", "uniqueness", "coherence", "fluency"]
    }
)

print(evaluation.app_link)